# Warm Start Testing

In [1]:
import numpy as np
import gurobipy as gp
import math
import sympy
import contextlib
import time
import os
from utils import result, METHODS, INF, EPS
from mps_reader_preprocessor import read_mps_preprocess
from polyhedral_model import PolyhedralModel
from polyhedron import Polyhedron

## Read in Polyhedron P

In [2]:
# mps_fn='C:\Users\DillW\OneDrive - The University of Colorado Denver\Documents\GitHub\simplex_like_alg\netlib_lp_subset\kb2'
problem_dir = 'netlib_lp_subset'
problem = 'adlittle'
mps_fn=os.path.join(problem_dir, problem)
results_dir='results'
max_time=300
sd_method='dual_simplex'
reset=False,
partition_polytope=False
n=0
k=0
spindle=False
spindle_dim=0
n_cone_facets=0
n_parallel_facets=0

In [3]:
### Build Initial Polyhedron from file
print(f'Reading {mps_fn}...')
c, B, d, A, b = read_mps_preprocess(mps_fn)
print('Building polyhedron...')
P = Polyhedron(B, d, A, b, c)

Reading netlib_lp_subset\adlittle...
Building polyhedron...
Problem size: n = 97,  m_B = 138,  m_A = 15


## Get Initial Feasible Solution

In [4]:
print('Finding feasible solution...')
x_feasible = P.find_feasible_solution(verbose=False)

Finding feasible solution...
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-01


## Perform SDCA

### Original Code

In [5]:
x = x_feasible
c=None
verbose=False
method='dual_simplex'
reset=False
max_time=300
first_warm_start=None
save_first_steps=0
problem_name=''

In [6]:
if c is not None:
    P.set_objective(c)

t0 = time.time()
x_current = x
if save_first_steps:
    np.save('solutions/{}_0.npy'.format(problem_name), x_current)      
active_inds = P.get_active_constraints(x_current)
    
pm = P.build_polyhedral_model(active_inds=active_inds, method=method)

if first_warm_start is not None:
    print('Using custom warm start')
    pm.set_solution(first_warm_start)
t1 = time.time()
build_time = t1 - t0
print('Polyhedral model build time: {}'.format(build_time))
    
sub_times = {'sd': [], 'step': [], 'solve': [], 'phase_times': []}    
descent_circuits = []
obj_values = []
step_sizes = []
iter_times = []
simplex_iters = []
iteration = 0
obj_value = P.c.dot(x_current)
obj_values.append(obj_value)
t2 = time.time()
iter_times.append(t2 - t1)


# compute steepest-descent direction
descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(verbose=verbose)
simplex_iters.append(num_steps)
sub_times['solve'].append(solve_time)
sub_times['phase_times'].append(phase_times)
    
t3 = time.time()
sub_times['sd'].append(t3 - t2)
    
while abs(steepness) > EPS:
        
    t3 = time.time()
    if reset:
        pm.reset()
        
    # take maximal step
    x_current, alpha, active_inds = P.take_maximal_step(descent_direction, y_pos, y_neg)  
        
    if iteration % 50 == 0 or iteration == 1:
        print('\nIteration {}'.format(iteration))
        print('Objective: {}'.format(obj_value))
        print('Steepness: {}'.format(steepness))
        print('Step length: {}'.format(alpha))
        
    t4 = time.time()
    obj_value = P.c.dot(x_current)
    obj_values.append(obj_value)
    iter_times.append(t4 - t1)
    sub_times['step'].append(t4 - t3) 
    descent_circuits.append(descent_direction)
    step_sizes.append(alpha)     
                
    if math.isinf(alpha):
        # problem is unbounded
        result(status=1, circuits=descent_circuits, steps=step_sizes)
        
    # compute steepest-descent direction
    pm.set_active_inds(active_inds)
    descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(
                                                                                                verbose=verbose)
        
    t5 = time.time()
    sub_times['sd'].append(t5 - t4)
    sub_times['solve'].append(solve_time)
    sub_times['phase_times'].append(phase_times)
    simplex_iters.append(num_steps)
        
    iteration += 1
    current_time = t5 - t1
    if current_time > max_time:
        result(status=2)
    if iteration <= save_first_steps:
        np.save('solutions/{}_{}.npy'.format(problem_name, iteration), x_current)

t6 = time.time()
total_time = t6 - t1   
print('Total time for steepest-descent scheme: {}'.format(total_time))

Building polyhedral model. Solve method: dual_simplex
Set parameter Method to value 1
Polyhedral model built!
Polyhedral model build time: 0.017157316207885742

Iteration 0
Objective: 671734.1100278501
Steepness: -1093.3333333333335
Step length: 1.492387497139124

Iteration 1
Objective: 670102.4330309781
Steepness: -608.695652173913
Step length: 14.047128014198497

Iteration 50
Objective: 225785.31769628637
Steepness: -1.1566308634677096
Step length: 10.693037152657151
Total time for steepest-descent scheme: 0.1264653205871582


### Updated Code

In [7]:
x = x_feasible
c=None
verbose=False
method='dual_simplex'
reset=False
max_time=300
first_warm_start=None
save_first_steps=0
problem_name=''

In [8]:
if c is not None:
    P.set_objective(c)
 
####get edge for initial circuit direction here#########
x_feasible_2= P.second_vert(x, verbose=False, record_objs=True)
init_edge = np.array(x_feasible_2) - np.array(x)
########################################################
### how do I get y_neg and y_pos for the while loop?

t0 = time.time()
x_current = x
if save_first_steps:
    np.save('solutions/{}_0.npy'.format(problem_name), x_current)      
active_inds = P.get_active_constraints(x_current)
    
pm = P.build_polyhedral_model(active_inds=active_inds, method=method)

first_warm_start = init_edge
if first_warm_start is not None:
    print('Using custom warm start')
    pm.set_solution(first_warm_start)
t1 = time.time()
build_time = t1 - t0
print('Polyhedral model build time: {}'.format(build_time))
    
sub_times = {'sd': [], 'step': [], 'solve': [], 'phase_times': []}    
descent_circuits = []
obj_values = []
step_sizes = []
iter_times = []
simplex_iters = []
iteration = 0
obj_value = P.c.dot(x_current)
obj_values.append(obj_value)
t2 = time.time()
iter_times.append(t2 - t1)


# compute steepest-descent direction
descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(verbose=verbose)
simplex_iters.append(num_steps)
sub_times['solve'].append(solve_time)
sub_times['phase_times'].append(phase_times)
    
t3 = time.time()
sub_times['sd'].append(t3 - t2)
    
while abs(steepness) > EPS:
        
    t3 = time.time()
    if reset:
        pm.reset()
        
    # take maximal step
    x_current, alpha, active_inds = P.take_maximal_step(descent_direction, y_pos, y_neg)  
        
    if iteration % 50 == 0 or iteration == 1:
        print('\nIteration {}'.format(iteration))
        print('Objective: {}'.format(obj_value))
        print('Steepness: {}'.format(steepness))
        print('Step length: {}'.format(alpha))
        
    t4 = time.time()
    obj_value = P.c.dot(x_current)
    obj_values.append(obj_value)
    iter_times.append(t4 - t1)
    sub_times['step'].append(t4 - t3) 
    descent_circuits.append(descent_direction)
    step_sizes.append(alpha)     
                
    if math.isinf(alpha):
        # problem is unbounded
        result(status=1, circuits=descent_circuits, steps=step_sizes)
        
    # compute steepest-descent direction
    pm.set_active_inds(active_inds)
    descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(
                                                                                                verbose=verbose)
        
    t5 = time.time()
    sub_times['sd'].append(t5 - t4)
    sub_times['solve'].append(solve_time)
    sub_times['phase_times'].append(phase_times)
    simplex_iters.append(num_steps)
        
    iteration += 1
    current_time = t5 - t1
    if current_time > max_time:
        result(status=2)
    if iteration <= save_first_steps:
        np.save('solutions/{}_{}.npy'.format(problem_name, iteration), x_current)

t6 = time.time()
total_time = t6 - t1   
print('Total time for steepest-descent scheme: {}'.format(total_time))

Feasible solution found with objective 671734.1100278501


AttributeError: 'CallbackClass' object has no attribute 'SOL'

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "src\\gurobipy\\callback.pxi", line 208, in gurobipy.CallbackClass.callback
  File "C:\Users\DillW\OneDrive - The University of Colorado Denver\Documents\GitHub\simplex_like_alg\polyhedron.py", line 548, in obj_callback
    x_optimal = model.cbGet(gp.GRB.Callback.SOL)
                            ^^^^^^^^^^^^^^^^^^^
AttributeError: 'CallbackClass' object has no attribute 'SOL'


AttributeError: 'CallbackClass' object has no attribute 'SOL'

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "src\\gurobipy\\callback.pxi", line 208, in gurobipy.CallbackClass.callback
  File "C:\Users\DillW\OneDrive - The University of Colorado Denver\Documents\GitHub\simplex_like_alg\polyhedron.py", line 548, in obj_callback
    x_optimal = model.cbGet(gp.GRB.Callback.SOL)
                            ^^^^^^^^^^^^^^^^^^^
AttributeError: 'CallbackClass' object has no attribute 'SOL'


...

AttributeError: 'CallbackClass' object has no attribute 'SOL'

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "src\\gurobipy\\callback.pxi", line 208, in gurobipy.CallbackClass.callback
  File "C:\Users\DillW\OneDrive - The University of Colorado Denver\Documents\GitHub\simplex_like_alg\polyhedron.py", line 548, in obj_callback
    x_optimal = model.cbGet(gp.GRB.Callback.SOL)
                            ^^^^^^^^^^^^^^^^^^^
AttributeError: 'CallbackClass' object has no attribute 'SOL'


AttributeError: 'CallbackClass' object has no attribute 'SOL'

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "src\\gurobipy\\callback.pxi", line 208, in gurobipy.CallbackClass.callback
  File "C:\Users\DillW\OneDrive - The University of Colorado Denver\Documents\GitHub\simplex_like_alg\polyhedron.py", line 548, in obj_callback
    x_optimal = model.cbGet(gp.GRB.Callback.SOL)
                            ^^^^^^^^^^^^^^^^^^^
AttributeError: 'CallbackClass' object has no attribute 'SOL'


NameError: name 'x_optimal' is not defined